# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys_mlopez import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [96]:
data = pd.read_csv("my_cities.csv")
data.head()

,Latitude,Longitude,City,Country,Max_Temp,Humidity,Wnd_Spd,Cloudiness,Dates,Pressure
0,-53.1500,-70.9167,Punta Arenas,CL,53.71,82,10.36,75,1644205844,1005
1,54.7183,135.3150,Chumikan,RU,0.86,78,13.82,15,1644205845,1020
2,-54.8000,-68.3000,Ushuaia,AR,60.46,67,14.97,40,1644205846,1005
3,-40.8135,-62.9967,Viedma,AR,60.89,77,8.57,0,1644205848,1019
4,-21.3585,55.7679,Saint-Philippe,RE,80.19,83,15.12,94,1644205849,1014


In [147]:
#Dropped Nulls
data_sub = data.dropna().reset_index(drop=True)
data_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558 entries, 0 to 557
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Latitude    558 non-null    float64
 1   Longitude   558 non-null    float64
 2   City        558 non-null    object 
 3   Country     558 non-null    object 
 4   Max_Temp    558 non-null    float64
 5   Humidity    558 non-null    int64  
 6   Wnd_Spd     558 non-null    float64
 7   Cloudiness  558 non-null    int64  
 8   Dates       558 non-null    int64  
 9   Pressure    558 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 43.7+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [88]:
#Accessing maps with API Key
gmaps.configure(api_key=g_key)

In [138]:
locations = data_sub.loc[:, ["Latitude", "Longitude"]]

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=data_sub['Humidity'],
    max_intensity=20, point_radius=7
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [90]:
#Max Temps <80 but >70, Wnd_Spd <10, & 0 cloudiness
mask = (data_sub.Max_Temp < 80) & (data_sub.Max_Temp > 70) & (data_sub.Wnd_Spd < 10) & (data_sub.Cloudiness == 0)

good_cities = data_sub.loc[mask]
good_cities

,Latitude,Longitude,City,Country,Max_Temp,Humidity,Wnd_Spd,Cloudiness,Dates,Pressure
5,-33.0153,27.9116,East London,ZA,70.74,94,4.61,0,1644205201,1014
91,-0.9000,-89.6000,Puerto Baquerizo Moreno,EC,73.27,81,9.91,0,1644205977,1014
116,18.8333,100.6667,Nan,TH,75.22,69,2.30,0,1644206020,1015
200,15.7432,-96.4661,Pochutla,MX,76.82,81,3.24,0,1644206146,1015
252,6.3163,8.1169,Abakaliki,NG,73.90,67,2.66,0,1644206221,1010


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [105]:
# coordinates for Pochutla - 15.7432	-96.4661
target_coordinates = f"{15.7432}, {-96.4661}"
target_search = ""

target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# Base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

places_data = response.json()

print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])
print(places_data["results"][0]["rating"])

Hotel Posada San Jose
Constitución, Centro, San Pedro Pochutla
3.8


In [116]:
lats = []
longs = []
hotel_names = []
addresses = []
cities = []
countries = []
ratings = []

for _, row in good_cities.iterrows():
    print(row.City)
    
    # coordinates
    target_coordinates = f"{row.Latitude}, {row.Longitude}"
    target_search = ""

    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    try:
        places_data = response.json()

        hotel = places_data["results"][0]["name"]
        address = places_data["results"][0]["vicinity"]
        rating = places_data["results"][0]["rating"]

    except Exception as e:
        print(e)
    
    # Put it in a list
    hotel_names.append(hotel)
    ratings.append(rating)
    addresses.append(address)
    cities.append(row.City)
    countries.append(row.Country)
    lats.append(row.Latitude)
    longs.append(row.Longitude)

East London
Puerto Baquerizo Moreno
Nan
'rating'
Pochutla
Abakaliki


In [117]:
hotel_df = pd.DataFrame()
hotel_df['City'] = cities
hotel_df['Country'] = countries
hotel_df['Latitude'] = lats
hotel_df['Longitude'] = longs
hotel_df['Hotel'] = hotel_names
hotel_df['Address'] = addresses
hotel_df['Rating'] = ratings

hotel_df

,City,Country,Latitude,Longitude,Hotel,Address,Rating
0,East London,ZA,-33.0153,27.9116,Tu Casa,"2 Ascot Road, Nahoon, East London",4.6
1,Puerto Baquerizo Moreno,EC,-0.9000,-89.6000,Hotel los Algarrobos,"Av Quito Y, Avenida Alsacio Northia, Puerto Ba...",4.1
2,Nan,TH,18.8333,100.6667,สะเนียน2,"RMQ6+429, Sanian, Mueang Nan District",4.1
3,Pochutla,MX,15.7432,-96.4661,Hotel Posada San Jose,"Constitución, Centro, San Pedro Pochutla",3.8
4,Abakaliki,NG,6.3163,8.1169,Grande Vitesse Apartments and Hotels,"5 Igwe Ogbuofia Street, Kpiri Kpiri (off Ogbag...",3.8


In [143]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Address</dt><dd>{Address}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Latitude", "Longitude"]]

In [144]:
hotel_df.head()

,City,Country,Latitude,Longitude,Hotel,Address,Rating
0,East London,ZA,-33.0153,27.9116,Tu Casa,"2 Ascot Road, Nahoon, East London",4.6
1,Puerto Baquerizo Moreno,EC,-0.9000,-89.6000,Hotel los Algarrobos,"Av Quito Y, Avenida Alsacio Northia, Puerto Ba...",4.1
2,Nan,TH,18.8333,100.6667,สะเนียน2,"RMQ6+429, Sanian, Mueang Nan District",4.1
3,Pochutla,MX,15.7432,-96.4661,Hotel Posada San Jose,"Constitución, Centro, San Pedro Pochutla",3.8
4,Abakaliki,NG,6.3163,8.1169,Grande Vitesse Apartments and Hotels,"5 Igwe Ogbuofia Street, Kpiri Kpiri (off Ogbag...",3.8


In [161]:
#Marker & Heat Layers
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
# heatmap_layer = gmaps.heatmap_layer(
#     hotel_locations, weights=data_sub['Humidity'],
#     max_intensity=20, point_radius=7
# )

fig = gmaps.figure()

#Layers
fig.add_layer(marker_layer)
fig.add_layer(heatmap_layer)

    
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Add marker layer ontop of heat map


# Display figure
